In [7]:
from pyproj import Proj, transform
import xml.etree.ElementTree as ET
import tqdm
import urllib.request
from PIL import Image
import pyproj

# Image quisation from Open Bayern Data
https://atlas.bayern.de/?c=670229,5335010&z=10&l=vt_luftbild,https%3A%2F%2Fgeodaten.bayern.de%2Fodd%2Fa%2Fdop40%2Fmeta%2Fkml%2Fgemeinde.kml&t=ba



In [2]:
import os
import urllib.request

# add folder path to project
folder_path = "C:\Projekte\TDS\TDS2324-TrafficAccidents\Data\Images\\"

# create folder if not exists
if not os.path.exists(folder_path):
    os.makedirs(folder_path)



# getting list of all urls
tree = ET.parse('C:\Projekte\TDS\TDS2324-TrafficAccidents\Data\metalink.meta4')
urls = []

for file_element in tree.findall('.//{urn:ietf:params:xml:ns:metalink}url'):

    if any(file_element.text.split("data")[1] in s for s in urls):

      continue

    else:

      urls.append(file_element.text)

# downloading all images
for url in tqdm.tqdm(urls):

  file_name = url.split("data")[1]

  if os.path.isfile(folder_path + file_name):

    continue

  else:

    urllib.request.urlretrieve(url, folder_path + file_name)




100%|██████████| 374/374 [33:40<00:00,  5.40s/it]


## Rename filesto ESPG4326
Since the Images are named after their coordinates in the referencesystem EPSG:25832 we are going to transform the coordinates in epsg:4326 and rename the images according to the new coordinates.

In [3]:
filepath = 'C:\\Projekte\\TDS\\TDS2324-TrafficAccidents\\Data\\Images'

# method to get coordinates from filename
def get_coordinates(filename):
  x = filename.split("_")[0]
  y = filename.split("_")[1].split(".")[0]

  # remove leading '32' from x and add 3 zeros
  x = x[2:]
  x = x + "000"

  # add 3 zeros to y
  y = y + "000"
  return x, y

# method to convert coordinates
def convert_coordinates(x, y):
  src_crs = Proj(init='epsg:25832') 
  dest_crs = Proj(init='epsg:4326')
  x = float(x)
  y = float(y)
  lat, lon = transform(src_crs,dest_crs,x,y) 
  return lon, lat

# iterate through ten first files in C:\Projekte\TDS\TDS2324-TrafficAccidents\Data\Images and rename it 
for i, filename in enumerate(os.listdir(filepath)):

  x, y = get_coordinates(filename)
  x2, y2 = convert_coordinates(x, y)
  file_to_rename = filepath + '\\' + filename
  new_file_name = filepath + '\\' + str(x2) + "_" + str(y2) + ".tif"
 # os.rename(file_to_rename, new_file_name)







c:\Users\miche\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\miche\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\miche\AppData\Local\Temp\ipykernel_22140\4006617107.py:22: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/got

Slicing the Pictures

Now we have the Pictures and their coordinates in the wgs84(EPSG 4326) referenc system. Next step is to slice the each picture from 1000m x 1000m in 40m x 40m pictures. Therefore we 
1. To name our now smaller images correctly we need a method that calculates us the coordinates of the new image, based on the original picture. 
    - Method calc_crs (org_long, org_lat, px_x, px_y)
    returns long, lat
2. Slice the original picture (2500px x 2500px) into 100px x 100px.


In [16]:
from math import cos
from math import radians

filepath_1km = 'C:\\Projekte\\TDS\\TDS2324-TrafficAccidents\\Data\\Images\\1km_img'
filepath_100m = 'C:\\Projekte\\TDS\\TDS2324-TrafficAccidents\\Data\\Images\\100m_img'


# get wgs84 coordinates from filename
def get_wgs_coordinates(filename):
    parts = filename.split('_')
    lat = parts[0]  # Latitude
    lon = parts[1].split('.')[0] + '.' + parts[1].split('.')[1]  # Longitude

    # Converting the extracted parts to floats
    lat_float = float(lat)
    lon_float = float(lon)
    return lat_float, lon_float





# iterate through C:\Projekte\TDS\TDS2324-TrafficAccidents\Data\Images\1km_img
for i, filename in enumerate(os.listdir(filepath_1km)):
    small_img_counter = 0

    lat,lon=get_wgs_coordinates(filename)

    print(lat,lon)
    print(filename)
    #create new folder for each image named after the img
   # new foldername = filename without .tif
    folder_name = filename.rsplit('.', 1)[0]
    folder_path = filepath_100m + '\\' + folder_name
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # cut image in 100px x 100px tiles
    img = Image.open(filepath_1km + '\\' + filename)
    width, height = img.size
    for x in range(0, width, 100):
        for y in range(0, height, 100):
            new_lat, new_lon = calc_crs(lat, lon, x, y)
            new_name = str(new_lat) + '_' + str(new_lon) + '.tif'
            
            box = (x, y, x+100, y+100)
            a = img.crop(box)
            a.save(folder_path + '\\' + new_name)
            small_img_counter += 1
    break



48.05955726486201 11.523250719974111
48.05955726486201_11.523250719974111.tif


In [ ]:
import os
from PIL import Image

# Pfadangaben
filepath_1km = 'C:\\Projekte\\TDS\\TDS2324-TrafficAccidents\\Data\\Images\\1km_img'
filepath_100m = 'C:\\Projekte\\TDS\\TDS2324-TrafficAccidents\\Data\\Images\\100m_img'

# Konstanten für die Berechnung
meters_per_degree_latitude = 111320  # Ungefährer Wert für Breitengrad in München
meters_per_degree_longitude = 74500  # Ungefährer Wert für Längengrad in München
pixel_size_m = 1000 / 2500  # Größe eines Pixels in Metern

# Funktion zur Berechnung der Koordinaten für jedes kleine Bild
def calc_crs(start_latitude, start_longitude, x_px, y_px):
    # Umrechnung von Pixelposition in Meter
    delta_x_m = x_px * pixel_size_m
    delta_y_m = y_px * pixel_size_m

    # Umrechnung von Meter in Grad
    delta_x_deg = delta_x_m / meters_per_degree_longitude
    delta_y_deg = delta_y_m / meters_per_degree_latitude

    # Neue Koordinaten
    new_longitude = start_longitude + delta_x_deg
    new_latitude = start_latitude + delta_y_deg

    return new_latitude, new_longitude

# Funktion zum Extrahieren der WGS84-Koordinaten aus dem Dateinamen
def get_wgs_coordinates(filename):
    parts = filename.split('_')
    lat = parts[0]  # Latitude
    lon = parts[1].split('.')[0] + '.' + parts[1].split('.')[1]  # Longitude

    # Umwandlung in Fließkommazahlen
    lat_float = float(lat)
    lon_float = float(lon)
    return lat_float, lon_float

# Durchlaufen der Bilder im 1km-Verzeichnis
for i, filename in enumerate(os.listdir(filepath_1km)):
    small_img_counter = 0

    lat, lon = get_wgs_coordinates(filename)

    # Erstellen eines neuen Ordners für jedes Bild
    folder_name = filename.rsplit('.', 1)[0]
    folder_path = os.path.join(filepath_100m, folder_name)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Zerschneiden des Bildes in 100px x 100px Kacheln
    img = Image.open(os.path.join(filepath_1km, filename))
    width, height = img.size
    for x in range(0, width, 100):
        for y in range(0, height, 100):
            new_lat, new_lon = calc_crs(lat, lon, x, y)
            new_name = f'{new_lat}_{new_lon}.tif'
            
            box = (x, y, x+100, y+100)
            a = img.crop(box)
            a.save(os.path.join(folder_path, new_name))
            small_img_counter += 1

    # Unterbrechen nach dem ersten Bild zur Demonstration
    break


In [ ]:
# Startkoordinaten (linke untere Ecke des großen Bildes)
start_latitude = 48.05955726486201
start_longitude = 11.523250719974111

# Größe des großen Bildes in Pixeln
total_width_px = 2500
total_height_px = 2500

# Größe des großen Bildes in Metern
total_width_m = 1000
total_height_m = 1000

# Umrechnungsfaktoren: Meter pro Grad
meters_per_degree_latitude = 111320  # Ungefährer Wert für Breitengrad in München
meters_per_degree_longitude = 74500  # Ungefährer Wert für Längengrad in München

# Größe der kleinen Bilder in Pixeln
small_image_size_px = 100

# Größe eines Pixels in Metern
pixel_size_m = total_width_m / total_width_px

# Berechnung der Koordinaten für jedes 100px x 100px Bild
coordinates = []
for y in range(0, total_height_px, small_image_size_px):
    for x in range(0, total_width_px, small_image_size_px):
        # Umrechnung von Pixelposition in Meter
        delta_x_m = x * pixel_size_m
        delta_y_m = y * pixel_size_m

        # Umrechnung von Meter in Grad
        delta_x_deg = delta_x_m / meters_per_degree_longitude
        delta_y_deg = delta_y_m / meters_per_degree_latitude

        # Hinzufügen der Deltas zu den Startkoordinaten
        new_longitude = start_longitude + delta_x_deg
        new_latitude = start_latitude + delta_y_deg

        # Speichern der Koordinaten der linken unteren Ecke jedes kleinen Bildes
        coordinates.append((new_latitude, new_longitude))

# Rückgabe einiger Beispielskoordinaten zur Überprüfung
coordinates[:5]  # Nur die ersten 5 Koordinaten als Beispiel

